In [ ]:
import habitat
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import time

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
from utils import init_config, FrameSkip, FrameStack, draw_top_down_map, RewardForwardFilter, RunningMeanStd,make_train_data
from agent import RNDAgent
from env import SimpleRLEnv

In [ ]:
config = init_config()

In [ ]:
env_basic = SimpleRLEnv(config=config)

In [ ]:
FRAMESKIP = 2
FRAMESTACK = 4
env = FrameSkip(env_basic, skip=FRAMESKIP)
env = FrameStack(env, FRAMESTACK, channel_order='chw')

In [ ]:
obs = env.reset()
positions = []
rotations = []

In [ ]:
obs,rew,done,info = env.step(2)
top_down_map = draw_top_down_map(info, env.env.env.obs["heading"][0], env.env.env.obs['rgb'].shape[0]) 
plt.imshow(top_down_map)
print(done, rew)
print(env.env.env.trux,env.env.env.truy)
print(env.env.env.goalx, env.env.env.goaly)
print(env.env.env.state['pos'])

In [ ]:
print(env.env.env.trux,env.env.env.truy)
print(env.env.env.goalx, env.env.env.goaly)

In [ ]:
FRAMESTACK = FRAMESTACK
output_size = 4
num_worker = 1
num_step = 500
gamma = 0.999
lam = 0.95
learning_rate = 1e-4
entropy_coef = 0.001
clip_grad_norm = 0.5
epoch = 100000000
mini_batch = 100
batch_size = int(num_step * num_worker / mini_batch)
ppo_eps = 0.1
int_gamma = 0.99
int_coef = 1.
ext_coef = 2.
use_cuda = True
use_gae = True

In [ ]:
agent = RNDAgent(
        FRAMESTACK,
        output_size,
        num_worker,
        num_step,
        gamma,
        lam=lam,
        learning_rate=learning_rate,
        ent_coef=entropy_coef,
        clip_grad_norm=clip_grad_norm,
        epoch=epoch,
        batch_size=batch_size,
        ppo_eps=ppo_eps,
        use_cuda=use_cuda,
        use_gae=use_gae
    )

In [ ]:
global_step = 0
global_update = 0
sample_env_idx = 0
sample_i_rall = 0
sample_rall = 0
sample_step = 0
sample_episode = 0
all_steps = 0
rew_trees = []
discounted_reward = RewardForwardFilter(int_gamma)
reward_rms = RunningMeanStd()
#wandb.watch(agent.model)
#wandb.watch(agent.rnd)
total_state_array, ext_target_array, int_target_array, total_action_array, total_adv_array, total_next_obs_array, total_policy_array = [], [], [], [], [], [], []

while True:
    states = env.reset()
    total_state, total_reward, total_done, total_next_state, total_action, total_int_reward, total_next_obs, total_ext_values, total_int_values, total_policy, total_policy_np = \
        [], [], [], [], [], [], [], [], [], [], []
    global_step += (num_worker * num_step)
    global_update += 1
    stps = 0
    dne = False
    
    for _ in range(num_step):
        actions, value_ext, value_int, policy = agent.get_action(states)
        intrinsic_reward = agent.compute_intrinsic_reward(states)
        s,r,d,i = env.step(actions[0])
        stps+=1    
        next_states, rewards, dones, real_dones, log_rewards, next_obs, intrinsic_rewards = [], [], [], [], [], [], []
        next_states.append(s)
        rewards.append(r)
        dones.append(d)
        next_obs.append(s)
        intrinsic_rewards.append(intrinsic_reward)
        
        next_states = np.stack(next_states)
        rewards = np.hstack(rewards)
        dones = np.hstack(dones)
        next_obs = np.stack(next_obs)
        intrinsic_rewards = np.hstack(intrinsic_reward)
        
        
        sample_i_rall += intrinsic_reward[sample_env_idx]
        total_next_obs.append(next_obs)
        total_int_reward.append(intrinsic_reward)
        total_state.append(states)
        total_reward.append(rewards)
        total_done.append(dones)
        total_action.append(actions)
        total_ext_values.append(value_ext)
        total_int_values.append(value_int)
        total_policy.append(policy)
        total_policy_np.append(policy.cpu().numpy())
        
        states = next_states[0]
        sample_rall += r
        sample_step += 1
        
        if d:
            dne = True
            sample_episode += 1
            sample_rall = 0
            
            sample_i_rall = 0
            
            break
    
    if not dne:
        sample_episode += 1
        sample_rall = 0
        sample_step = 0
        sample_i_rall = 0
        
    
    rew_trees.append(sum(sum(total_reward)))
    all_steps += sample_step
    print('\t',sample_step,rew_trees[-1],np.mean(rew_trees[-100:]),i['spl'],all_steps)
    #wandb.log({"Steps in one run": sample_step, "Episode reward": rew_trees[-1], "Episode SPL": i['spl'], "All steps": all_steps})
    sample_step = 0
    _, value_ext, value_int, _ = agent.get_action(states)
    total_ext_values.append(value_ext)
    total_int_values.append(value_int) 
    
    total_state = np.stack(total_state)
    total_reward = np.stack(total_reward).transpose().clip(-1, 1)
    total_action = np.stack(total_action).transpose().reshape([-1])
    total_done = np.stack(total_done).transpose()
    total_next_obs = np.stack(total_next_obs)
    total_ext_values = np.stack(total_ext_values).transpose()
    total_int_values = np.stack(total_int_values).transpose()
    total_logging_policy = np.vstack(total_policy_np)
    
    total_int_reward = np.stack(total_int_reward).transpose()
    total_reward_per_env = np.array([discounted_reward.update(reward_per_step) for reward_per_step in
                                     total_int_reward.T])
    mean, std, count = np.mean(total_reward_per_env), np.std(total_reward_per_env), len(total_reward_per_env)
    reward_rms.update_from_moments(mean, std ** 2, count)
    total_int_reward /= np.sqrt(reward_rms.var)
    
    ext_target, ext_adv = make_train_data(total_reward,
                                          total_done,
                                          total_ext_values.reshape((1,-1)),
                                          gamma,
                                          stps,
                                          num_worker,
                                          use_gae,
                                          lam)
    # None Episodic
    int_target, int_adv = make_train_data(total_int_reward,
                                          np.zeros_like(total_int_reward),
                                          total_int_values.reshape((1,-1)),
                                          int_gamma,
                                          stps,
                                          num_worker,
                                          use_gae,
                                          lam)
    
    
    total_adv = int_adv * int_coef + ext_adv * ext_coef
    print(global_update,'\t',sum(total_adv),sum(ext_adv),sum(int_adv))
    #wandb.log({"Episode Total Reward": sum(total_adv), "Episode Ext Reward": sum(ext_adv), "Episode Int Reward": sum(int_adv)})
    st = time.time()
    
    
    
    total_state_array.extend(total_state)
    ext_target_array.extend(ext_target)
    int_target_array.extend(int_target)
    total_action_array.extend(total_action)
    total_adv_array.extend(total_adv)
    total_next_obs_array.extend(total_next_obs)
    total_policy_array.extend(total_policy)
    
    if len(total_state_array)>400:
        agent.train_model(total_state_array, ext_target_array, int_target_array, total_action_array,
                      total_adv_array, total_next_obs_array,
                      total_policy_array)
        total_state_array, ext_target_array, int_target_array, total_action_array, total_adv_array, total_next_obs_array, total_policy_array = [], [], [], [], [], [], []

    end = time.time()
    print("Time to train: ",end-st)